In [64]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

In [65]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
import random

In [66]:
tf.keras.backend.clear_session()

In [67]:
subject = 'Rio Grande Do Norte - Consumo de Cimento (t)'
start_index = 0
split_index = 191 #Referente aos 230 anos de input 
window_size = 36
train_split = split_index + 1 - 30

In [68]:
data = pd.read_csv('2003_mo_model_input_RN.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Unnamed: 0,Rio Grande do Norte - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Rio Grande Do Norte - Produção de Cimento (t),Rio Grande do Norte - PIB - Estadual,Rio Grande do Norte - PIB - Construção Civil,Rio Grande do Norte - PIB - Per Capita,Rio Grande do Norte - PIB - Preços de Mercado,Rio Grande do Norte - Desemprego,Rio Grande Do Norte - Consumo de Cimento (t)
0,2003-1,0.689488,0.724032,11.520143,1.639718,1.036534,1.772069e+08,7.330309e+06,0.969649,27.698760,3.164847e+07,1.998664e+06,9.097985,2.879532e+07,8.294170,36.825
1,2003-2,0.689892,0.690297,11.189862,1.378899,0.993449,1.773884e+08,7.335910e+06,0.950783,27.407645,3.168897e+07,2.001107e+06,9.101271,2.881033e+07,8.288224,34.088
2,2003-3,0.690297,0.669681,10.820792,1.924317,0.973020,1.775699e+08,7.341511e+06,0.938332,27.377422,3.172947e+07,2.003550e+06,9.104557,2.882535e+07,8.282278,30.150
3,2003-4,0.690702,0.660494,10.417840,1.331174,0.940489,1.777514e+08,7.347112e+06,0.926401,27.542280,3.176997e+07,2.005994e+06,9.107843,2.884036e+07,8.276332,31.224
4,2003-5,0.691107,0.648337,9.959690,1.736072,0.917493,1.779329e+08,7.352713e+06,0.951683,27.944900,3.181046e+07,2.008437e+06,9.111128,2.885538e+07,8.270387,33.606
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,2022-8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,95.557326,NaN,NaN,NaN,NaN,NaN,83.890
236,2022-9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,94.910014,NaN,NaN,NaN,NaN,NaN,83.438
237,2022-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,94.800783,NaN,NaN,NaN,NaN,NaN,80.005
238,2022-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,94.752983,NaN,NaN,NaN,NaN,NaN,75.441


In [69]:
input_data = data.iloc[:split_index + 1,1:-1]
mean = np.mean(input_data, axis=0)
stddev =  np.std(input_data, axis=0)
input_data = ((input_data - mean) /stddev)
# scaler=MinMaxScaler()
# scaled_data = scaler.fit_transform(input_data)
# input_data = pd.DataFrame(scaled_data, columns=input_data.columns)
input_data

,Rio Grande do Norte - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Rio Grande Do Norte - Produção de Cimento (t),Rio Grande do Norte - PIB - Estadual,Rio Grande do Norte - PIB - Construção Civil,Rio Grande do Norte - PIB - Per Capita,Rio Grande do Norte - PIB - Preços de Mercado,Rio Grande do Norte - Desemprego
0,-2.184058,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,-1.183161,-1.731345,-0.778207,-2.784513,-2.193066,-0.876481
1,-2.147573,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,-1.192364,-1.711183,-0.728728,-2.703361,-2.146603,-0.878896
2,-2.111089,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,-1.193319,-1.691021,-0.679248,-2.622210,-2.100140,-0.881311
3,-2.074605,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,-1.188108,-1.670859,-0.629768,-2.541059,-2.053677,-0.883727
4,-2.038120,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,-1.175380,-1.650697,-0.580289,-2.459908,-2.007214,-0.886142
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,1.321156,-2.010387,-0.572934,-1.281958,0.589021,1.365911,0.389193,-1.749976,0.389536,1.102751,-1.573216,0.457957,0.775635,1.052347
188,1.314269,-1.870713,-0.588777,-1.358588,1.043728,1.376610,0.370392,-1.593005,0.398528,1.092237,-1.561825,0.427807,0.759230,1.051753
189,1.307381,-1.806230,-0.612606,-1.511565,1.387010,1.387308,0.351592,-1.351489,0.414978,1.081724,-1.550435,0.397658,0.742826,1.051160
190,1.300494,-1.727496,-0.640956,-1.421708,1.815728,1.398006,0.332791,-1.198492,0.429834,1.071210,-1.539044,0.367509,0.726421,1.050566


In [70]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-12)
target_data

0      32.845
1      26.809
2      34.465
3      30.563
4      32.574
        ...  
235       NaN
236       NaN
237       NaN
238       NaN
239       NaN
Name: Rio Grande Do Norte - Consumo de Cimento (t), Length: 240, dtype: float64

In [71]:
# input para treinamento
train_input = input_data.iloc[start_index:train_split]
train_input

,Rio Grande do Norte - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Rio Grande Do Norte - Produção de Cimento (t),Rio Grande do Norte - PIB - Estadual,Rio Grande do Norte - PIB - Construção Civil,Rio Grande do Norte - PIB - Per Capita,Rio Grande do Norte - PIB - Preços de Mercado,Rio Grande do Norte - Desemprego
0,-2.184058,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,-1.183161,-1.731345,-0.778207,-2.784513,-2.193066,-0.876481
1,-2.147573,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,-1.192364,-1.711183,-0.728728,-2.703361,-2.146603,-0.878896
2,-2.111089,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,-1.193319,-1.691021,-0.679248,-2.622210,-2.100140,-0.881311
3,-2.074605,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,-1.188108,-1.670859,-0.629768,-2.541059,-2.053677,-0.883727
4,-2.038120,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,-1.175380,-1.650697,-0.580289,-2.459908,-2.007214,-0.886142
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,1.384282,-0.214006,-0.607704,0.059280,-1.233012,1.031384,0.819304,-0.883659,1.046207,1.183768,-1.442416,0.625176,0.985411,1.213189
158,1.382302,-0.434717,-0.620523,0.222389,-1.299304,1.042716,0.808136,-0.950771,1.000373,1.187707,-1.461501,0.642268,0.990347,1.201976
159,1.380323,-0.524091,-0.631530,0.191929,-1.248662,1.054049,0.796969,-1.028465,0.957199,1.191647,-1.480587,0.659360,0.995283,1.190764
160,1.378343,-0.614500,-0.640320,0.385687,-1.068274,1.065381,0.785801,-1.103668,0.908410,1.195587,-1.499673,0.676453,1.000219,1.179551


In [72]:
# Alvo para treinamento
train_target = target_data.iloc[start_index:train_split]
train_target

0      32.845
1      26.809
2      34.465
3      30.563
4      32.574
        ...  
157    52.722
158    63.109
159    49.356
160    55.725
161    50.796
Name: Rio Grande Do Norte - Consumo de Cimento (t), Length: 162, dtype: float64

In [73]:
def create_batches(t_input, t_target, window_size, start_from):
    
    X_batches = []
    y_batches = []

    train_input_values = t_input.values 

    for i in range(len(t_input) - window_size):
        
        X_window = train_input_values[i:i+window_size, :]
        y_target = t_target[start_from+i+window_size]

        X_batches.append(X_window)
        y_batches.append(y_target)

    return np.array(X_batches), np.array(y_batches)

In [74]:
reshaped_train, reshaped_target = create_batches(train_input, 
                                                 train_target, 
                                                 window_size, 
                                                 start_index)
reshaped_train.shape

(126, 36, 14)

In [75]:
# input para treinamento
test_input = input_data.iloc[train_split - window_size:split_index + 1]
test_input

,Rio Grande do Norte - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Rio Grande Do Norte - Produção de Cimento (t),Rio Grande do Norte - PIB - Estadual,Rio Grande do Norte - PIB - Construção Civil,Rio Grande do Norte - PIB - Per Capita,Rio Grande do Norte - PIB - Preços de Mercado,Rio Grande do Norte - Desemprego
126,0.458594,0.888984,-0.460555,-1.131704,-0.368821,0.651397,0.944085,0.045243,1.480898,0.826780,0.124108,0.924042,0.982707,1.063612
127,0.463887,0.954254,-0.440372,-0.613483,-0.328087,0.664707,0.947319,0.061828,1.485581,0.843350,0.061862,0.903284,0.982590,1.075867
128,0.469179,1.045217,-0.419247,-0.542919,-0.176031,0.678017,0.950553,0.046225,1.491421,0.859920,-0.000383,0.882526,0.982474,1.088122
129,0.474472,1.176395,-0.397019,-1.334517,-0.113037,0.691327,0.953786,0.032522,1.490890,0.876489,-0.062629,0.861768,0.982357,1.100376
130,0.479764,1.303259,-0.376532,-1.511762,-0.022703,0.704637,0.957020,0.042757,1.486995,0.893059,-0.124874,0.841010,0.982241,1.112631
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,1.321156,-2.010387,-0.572934,-1.281958,0.589021,1.365911,0.389193,-1.749976,0.389536,1.102751,-1.573216,0.457957,0.775635,1.052347
188,1.314269,-1.870713,-0.588777,-1.358588,1.043728,1.376610,0.370392,-1.593005,0.398528,1.092237,-1.561825,0.427807,0.759230,1.051753
189,1.307381,-1.806230,-0.612606,-1.511565,1.387010,1.387308,0.351592,-1.351489,0.414978,1.081724,-1.550435,0.397658,0.742826,1.051160
190,1.300494,-1.727496,-0.640956,-1.421708,1.815728,1.398006,0.332791,-1.198492,0.429834,1.071210,-1.539044,0.367509,0.726421,1.050566


In [76]:
reshaped_test, reshaped_test_target = create_batches(test_input, 
                                                     target_data, 
                                                     window_size, 
                                                     train_split - window_size)
reshaped_test.shape 

(30, 36, 14)

In [77]:
def validation_splitter(arr, div_factor, add_factor=0):
    split_factor = len(arr) // div_factor
    positions_to_drop_index = []
    positions_to_drop = []
    for i in range(split_factor):
        pos = len(arr) - (i * div_factor + 1)
        positions_to_drop_index.append(pos)
        positions_to_drop.append(pos + add_factor)
    
    arr_droped = arr[positions_to_drop]
    arr_result = np.delete(arr, positions_to_drop_index, axis=0)
    
    return arr_result, arr_droped

In [78]:
# Rede Neural Recorrente com optmizador Estocástico
def bidirectional_lstm_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    train, train_val = validation_splitter(train_input, 7)
    target,target_val = validation_splitter(train_target, 7)
#     display(train.shape)
#     display(train_val.shape)
#     display(target.shape)
#     display(target_val.shape)
    # Método estocástico e learning rate=0.005
#     optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Bidirectional(
            tf.keras.layers.LSTM(36, activation='tanh', 
                                 return_sequences=False), 
                                 input_shape=(train_input.shape[1], 
                                              train_input.shape[2])),
#         tf.keras.layers.Dropout(0.5),
#         tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(180, activation='tanh')),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                  loss='mean_squared_error')   
    history = model.fit(train, 
                        target, 
                        epochs=10000,
                        validation_data=(train_val,
                                         target_val),
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [79]:
def get_a_good_seed(train_input, train_target, test_input, test_target):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(20)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = bidirectional_lstm_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [80]:
winner_seed = get_a_good_seed(reshaped_train, reshaped_target, reshaped_test, reshaped_test_target)
print(f"\n\nfinal_seed: {winner_seed}")

[2350745573, 3927429384, 1401934291, 1704849978, 973234399, 2242698770, 213402327, 1261419371, 1322089802, 1846239336, 3983470112, 4269015352, 1537127313, 3558314880, 1243866309, 2175824288, 1489462026, 136934144, 3786026734, 3911262856]


Step: 0 ___________________________________________
val_loss: 26.623945236206055
winner_seed: 2350745573


Step: 1 ___________________________________________
val_loss: 14.80256175994873
winner_seed: 3927429384


Step: 2 ___________________________________________
val_loss: 28.762346267700195


Step: 3 ___________________________________________
val_loss: 12.72201919555664
winner_seed: 1704849978


Step: 4 ___________________________________________
val_loss: 16.49707794189453


Step: 5 ___________________________________________
val_loss: 15.388569831848145


Step: 6 ___________________________________________
val_loss: 16.563426971435547


Step: 7 ___________________________________________
val_loss: 17.018245697021484


Step: 8 ___________________

In [81]:
trained_model, history = bidirectional_lstm_model(reshaped_train, 
                                                  reshaped_target, 
                                                  want_verbose=1, 
                                                  seed=winner_seed)

Epoch 1/10000
4/4 [==============================] - 3s 135ms/step - loss: 4973.0693 - val_loss: 4906.6636
Epoch 2/10000
4/4 [==============================] - 0s 10ms/step - loss: 4909.9473 - val_loss: 4844.3828
Epoch 3/10000
4/4 [==============================] - 0s 10ms/step - loss: 4845.6694 - val_loss: 4774.9473
Epoch 4/10000
4/4 [==============================] - 0s 12ms/step - loss: 4773.7295 - val_loss: 4697.5269
Epoch 5/10000
4/4 [==============================] - 0s 12ms/step - loss: 4693.0605 - val_loss: 4612.1929
Epoch 6/10000
4/4 [==============================] - 0s 11ms/step - loss: 4603.1807 - val_loss: 4519.4756
Epoch 7/10000
4/4 [==============================] - 0s 10ms/step - loss: 4507.7852 - val_loss: 4419.8955
Epoch 8/10000
4/4 [==============================] - 0s 11ms/step - loss: 4407.9502 - val_loss: 4314.9004
Epoch 9/10000
4/4 [==============================] - 0s 10ms/step - loss: 4300.6841 - val_loss: 4206.9873
Epoch 10/10000
4/4 [=========================

4/4 [==============================] - 0s 10ms/step - loss: 1407.0592 - val_loss: 1373.0613
Epoch 78/10000
4/4 [==============================] - 0s 11ms/step - loss: 1389.2911 - val_loss: 1355.0607
Epoch 79/10000
4/4 [==============================] - 0s 10ms/step - loss: 1371.5518 - val_loss: 1337.3601
Epoch 80/10000
4/4 [==============================] - 0s 10ms/step - loss: 1353.7725 - val_loss: 1319.9816
Epoch 81/10000
4/4 [==============================] - 0s 11ms/step - loss: 1336.3224 - val_loss: 1302.7570
Epoch 82/10000
4/4 [==============================] - 0s 12ms/step - loss: 1318.9456 - val_loss: 1285.7928
Epoch 83/10000
4/4 [==============================] - 0s 11ms/step - loss: 1302.2585 - val_loss: 1268.9606
Epoch 84/10000
4/4 [==============================] - 0s 10ms/step - loss: 1285.6812 - val_loss: 1252.4828
Epoch 85/10000
4/4 [==============================] - 0s 11ms/step - loss: 1268.9442 - val_loss: 1236.3113
Epoch 86/10000
4/4 [==============================] 

4/4 [==============================] - 0s 11ms/step - loss: 522.7091 - val_loss: 497.9855
Epoch 155/10000
4/4 [==============================] - 0s 9ms/step - loss: 516.3152 - val_loss: 491.6272
Epoch 156/10000
4/4 [==============================] - 0s 9ms/step - loss: 509.8685 - val_loss: 485.3941
Epoch 157/10000
4/4 [==============================] - 0s 9ms/step - loss: 503.6564 - val_loss: 479.2218
Epoch 158/10000
4/4 [==============================] - 0s 9ms/step - loss: 497.6783 - val_loss: 473.0954
Epoch 159/10000
4/4 [==============================] - 0s 9ms/step - loss: 491.3465 - val_loss: 467.1971
Epoch 160/10000
4/4 [==============================] - 0s 10ms/step - loss: 485.5549 - val_loss: 461.3166
Epoch 161/10000
4/4 [==============================] - 0s 9ms/step - loss: 479.7722 - val_loss: 455.5230
Epoch 162/10000
4/4 [==============================] - 0s 10ms/step - loss: 473.9425 - val_loss: 449.7855
Epoch 163/10000
4/4 [==============================] - 0s 10ms/step 

4/4 [==============================] - 0s 11ms/step - loss: 238.3748 - val_loss: 218.3417
Epoch 232/10000
4/4 [==============================] - 0s 11ms/step - loss: 235.7797 - val_loss: 215.8641
Epoch 233/10000
4/4 [==============================] - 0s 11ms/step - loss: 233.7473 - val_loss: 214.0469
Epoch 234/10000
4/4 [==============================] - 0s 11ms/step - loss: 231.8344 - val_loss: 212.7026
Epoch 235/10000
4/4 [==============================] - 0s 11ms/step - loss: 230.0921 - val_loss: 211.1463
Epoch 236/10000
4/4 [==============================] - 0s 12ms/step - loss: 228.3445 - val_loss: 209.3241
Epoch 237/10000
4/4 [==============================] - 0s 11ms/step - loss: 226.6105 - val_loss: 207.2999
Epoch 238/10000
4/4 [==============================] - 0s 10ms/step - loss: 224.4714 - val_loss: 205.1193
Epoch 239/10000
4/4 [==============================] - 0s 10ms/step - loss: 221.7788 - val_loss: 203.5082
Epoch 240/10000
4/4 [==============================] - 0s 10ms

4/4 [==============================] - 0s 12ms/step - loss: 116.3817 - val_loss: 110.6805
Epoch 309/10000
4/4 [==============================] - 0s 11ms/step - loss: 115.5719 - val_loss: 109.1432
Epoch 310/10000
4/4 [==============================] - 0s 11ms/step - loss: 114.6504 - val_loss: 108.8772
Epoch 311/10000
4/4 [==============================] - 0s 11ms/step - loss: 113.8629 - val_loss: 108.9810
Epoch 312/10000
4/4 [==============================] - 0s 11ms/step - loss: 113.2407 - val_loss: 107.3630
Epoch 313/10000
4/4 [==============================] - 0s 10ms/step - loss: 112.8650 - val_loss: 106.0163
Epoch 314/10000
4/4 [==============================] - 0s 11ms/step - loss: 111.1206 - val_loss: 106.2075
Epoch 315/10000
4/4 [==============================] - 0s 11ms/step - loss: 112.8129 - val_loss: 101.0953
Epoch 316/10000
4/4 [==============================] - 0s 11ms/step - loss: 113.7880 - val_loss: 94.9753
Epoch 317/10000
4/4 [==============================] - 0s 11ms/

...

Epoch 2510/10000
4/4 [==============================] - 0s 11ms/step - loss: 6.7967 - val_loss: 19.1753
Epoch 2511/10000
4/4 [==============================] - 0s 10ms/step - loss: 6.7934 - val_loss: 20.9034
Epoch 2512/10000
4/4 [==============================] - 0s 10ms/step - loss: 6.1627 - val_loss: 21.5893
Epoch 2513/10000
4/4 [==============================] - 0s 10ms/step - loss: 6.1671 - val_loss: 19.8908
Epoch 2514/10000
4/4 [==============================] - 0s 10ms/step - loss: 8.1799 - val_loss: 19.8017
Epoch 2515/10000
4/4 [==============================] - 0s 11ms/step - loss: 6.5942 - val_loss: 23.4008
Epoch 2516/10000
4/4 [==============================] - 0s 11ms/step - loss: 6.5463 - val_loss: 26.0947
Epoch 2517/10000
4/4 [==============================] - 0s 11ms/step - loss: 6.8559 - val_loss: 22.7019
Epoch 2518/10000
4/4 [==============================] - 0s 11ms/step - loss: 6.6000 - val_loss: 21.3676
Epoch 2519/10000
4/4 [==============================] - 0s 13ms/

Epoch 2589/10000
4/4 [==============================] - 0s 11ms/step - loss: 5.2498 - val_loss: 18.8755
Epoch 2590/10000
4/4 [==============================] - 0s 11ms/step - loss: 5.1812 - val_loss: 17.7248
Epoch 2591/10000
4/4 [==============================] - 0s 12ms/step - loss: 5.4490 - val_loss: 17.1217
Epoch 2592/10000
4/4 [==============================] - 0s 11ms/step - loss: 5.2768 - val_loss: 18.7073
Epoch 2593/10000
4/4 [==============================] - 0s 12ms/step - loss: 5.2296 - val_loss: 18.9465
Epoch 2594/10000
4/4 [==============================] - 0s 10ms/step - loss: 5.2951 - val_loss: 19.0228
Epoch 2595/10000
4/4 [==============================] - 0s 11ms/step - loss: 5.2722 - val_loss: 19.0538
Epoch 2596/10000
4/4 [==============================] - 0s 11ms/step - loss: 5.2588 - val_loss: 18.8456
Epoch 2597/10000
4/4 [==============================] - 0s 10ms/step - loss: 5.2724 - val_loss: 18.6566
Epoch 2598/10000
4/4 [==============================] - 0s 11ms/

Epoch 2668/10000
4/4 [==============================] - 0s 9ms/step - loss: 4.5990 - val_loss: 23.0201
Epoch 2669/10000
4/4 [==============================] - 0s 10ms/step - loss: 5.1367 - val_loss: 23.9631
Epoch 2670/10000
4/4 [==============================] - 0s 10ms/step - loss: 5.1345 - val_loss: 23.6909
Epoch 2671/10000
4/4 [==============================] - 0s 9ms/step - loss: 4.9339 - val_loss: 23.1557
Epoch 2672/10000
4/4 [==============================] - 0s 10ms/step - loss: 4.7226 - val_loss: 22.5635
Epoch 2673/10000
4/4 [==============================] - 0s 11ms/step - loss: 4.7741 - val_loss: 21.2549
Epoch 2674/10000
4/4 [==============================] - 0s 10ms/step - loss: 5.0921 - val_loss: 20.8240
Epoch 2675/10000
4/4 [==============================] - 0s 11ms/step - loss: 4.7631 - val_loss: 22.3798
Epoch 2676/10000
4/4 [==============================] - 0s 10ms/step - loss: 4.9479 - val_loss: 24.5614
Epoch 2677/10000
4/4 [==============================] - 0s 10ms/st

Epoch 2747/10000
4/4 [==============================] - 0s 10ms/step - loss: 5.3507 - val_loss: 17.7037
Epoch 2748/10000
4/4 [==============================] - 0s 11ms/step - loss: 5.5040 - val_loss: 16.9462
Epoch 2749/10000
4/4 [==============================] - 0s 10ms/step - loss: 5.3775 - val_loss: 19.1380
Epoch 2750/10000
4/4 [==============================] - 0s 11ms/step - loss: 5.3612 - val_loss: 17.3284
Epoch 2751/10000
4/4 [==============================] - 0s 10ms/step - loss: 5.3182 - val_loss: 16.2668
Epoch 2752/10000
4/4 [==============================] - 0s 10ms/step - loss: 5.1430 - val_loss: 17.5553
Epoch 2753/10000
4/4 [==============================] - 0s 10ms/step - loss: 5.8523 - val_loss: 18.8060
Epoch 2754/10000
4/4 [==============================] - 0s 10ms/step - loss: 5.4531 - val_loss: 19.2612
Epoch 2755/10000
4/4 [==============================] - 0s 10ms/step - loss: 5.0052 - val_loss: 17.2787
Epoch 2756/10000
4/4 [==============================] - 0s 10ms/

Epoch 2826/10000
4/4 [==============================] - 0s 13ms/step - loss: 4.7468 - val_loss: 16.4795
Epoch 2827/10000
4/4 [==============================] - 0s 10ms/step - loss: 5.0720 - val_loss: 16.1642
Epoch 2828/10000
4/4 [==============================] - 0s 11ms/step - loss: 4.8095 - val_loss: 16.8683
Epoch 2829/10000
4/4 [==============================] - 0s 11ms/step - loss: 5.2189 - val_loss: 17.0189
Epoch 2830/10000
4/4 [==============================] - 0s 9ms/step - loss: 5.0082 - val_loss: 16.9743
Epoch 2831/10000
4/4 [==============================] - 0s 10ms/step - loss: 4.8411 - val_loss: 17.3525
Epoch 2832/10000
4/4 [==============================] - 0s 9ms/step - loss: 4.9531 - val_loss: 17.5802
Epoch 2833/10000
4/4 [==============================] - 0s 10ms/step - loss: 4.6997 - val_loss: 17.4750
Epoch 2834/10000
4/4 [==============================] - 0s 10ms/step - loss: 4.8422 - val_loss: 17.8417
Epoch 2835/10000
4/4 [==============================] - 0s 10ms/st

In [82]:
def mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    results_data = []

    for i in range(len(test_target)):
        prediction = model.predict(test_input[i:i+1])
        target = test_target[start_target + i]
        error = np.abs(prediction - target)
        errors.append(error)
        error_percent.append(error/target)
        results_data.append([f"Month-{i + 1}", 
                             prediction[0][0], 
                             target, 
                             error[0][0]])

    df_results = pd.DataFrame(results_data, columns=["Month", "Prediction", "Target", "Error"])

    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return df_results, mae, mape

In [83]:
df_results, mae, mape = mae_mape_calculator(trained_model, 
                                            reshaped_test, 
                                            reshaped_test_target, 
                                            start_index)
pd.set_option('display.max_columns', None)
df_results.T

1/1 [==============================] - 0s 15ms/step


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
Month,Month-1,Month-2,Month-3,Month-4,Month-5,Month-6,Month-7,Month-8,Month-9,Month-10,Month-11,Month-12,Month-13,Month-14,Month-15,Month-16,Month-17,Month-18,Month-19,Month-20,Month-21,Month-22,Month-23,Month-24,Month-25,Month-26,Month-27,Month-28,Month-29,Month-30
Prediction,53.273338,51.76965,49.737476,47.582256,45.31794,43.247559,41.577152,40.66288,39.991116,39.578869,38.833706,38.243572,37.787579,37.493416,37.394398,37.268948,37.258595,34.031242,33.783657,33.559151,33.29388,32.755741,32.086391,30.978798,30.241282,29.191776,27.964384,26.491293,25.339933,24.4671
Target,48.75,59.738,60.303,51.009,48.874,48.506,42.721,32.801,37.638,37.358,31.339,45.311,42.085,48.52,41.365,43.813,43.816,38.245,46.081,38.703,36.037,38.43,44.471,37.884,51.585,54.306,52.511,59.472,54.022,49.791
Error,4.523338,7.968349,10.565525,3.426743,3.556061,5.258442,1.143848,7.861881,2.353115,2.220867,7.494705,7.067429,4.297421,11.026585,3.970604,6.544052,6.557407,4.213757,12.297344,5.143848,2.743118,5.674259,12.384609,6.905201,21.343718,25.114223,24.546618,32.980705,28.682066,25.3239


In [84]:
display(mae)
display(mape)

10.106325

0.20966366

In [85]:
def year_mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    
    target_sum = 0
    prediction_sum = 0
    
    for i in range(len(test_target)):
        if i % 12 == 0 and i != 0:
            error = np.abs(target_sum - prediction_sum)
            errors.append(error)
            error_percent.append(error / target_sum)
            print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
            target_sum = 0
            prediction_sum = 0
            
        prediction = model.predict(test_input[i:i+1])
        target_sum += test_target[start_target + i]
        prediction_sum += prediction
        
    error = np.abs(target_sum - prediction_sum)
    errors.append(error)
    error_percent.append(error / target_sum)
    print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
        
    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return errors, mae, mape

In [86]:
errors, mae, mape = year_mae_mape_calculator(trained_model, 
                                             reshaped_test, 
                                             reshaped_test_target, 
                                             start_index)
display(errors)
display(mae)
display(mape)

1/1 [==============================] - 0s 14ms/step
Ano-0: |Prediction[[529.8155]] - Target[544.3480000000001]| =  Error: [[14.532532]]; MAPE:[[0.02669713]]
1/1 [==============================] - 0s 14ms/step
Ano-0: |Prediction[[417.69177]] - Target[499.45000000000005]| =  Error: [[81.75824]]; MAPE:[[0.16369654]]
1/1 [==============================] - 0s 16ms/step
Ano-5: |Prediction[[163.69577]] - Target[321.687]| =  Error: [[157.99124]]; MAPE:[[0.49113342]]


[array([[14.532532]], dtype=float32),
 array([[81.75824]], dtype=float32),
 array([[157.99124]], dtype=float32)]

84.76067

0.2271757